# Projeto 3

<b>Tema:</b> Estimar o retorno financeiro de um filme

<b>Técnica:</b> Regressão Linear

<b>Grupo:</b> Lucas Leal Vale, Matheus Augusto Soares, Rafael Almada
___

In [9]:
import pandas as pd
import numpy as np
from scipy import stats
import matplotlib.pyplot as plt
%matplotlib inline

In [10]:
movies=pd.read_csv('movies_metadata.csv', low_memory=False)
imdb=pd.read_csv('ratings_small.csv',low_memory=False)

In [11]:
movies.columns

Index(['adult', 'belongs_to_collection', 'budget', 'genres', 'homepage', 'id',
       'imdb_id', 'original_language', 'original_title', 'overview',
       'popularity', 'poster_path', 'production_companies',
       'production_countries', 'release_date', 'revenue', 'runtime',
       'spoken_languages', 'status', 'tagline', 'title', 'video',
       'vote_average', 'vote_count'],
      dtype='object')

In [12]:
df=movies.loc[:, {'id','original_title','original_language','genres','popularity','runtime','budget','revenue'}]
df.head()

,original_language,popularity,runtime,budget,genres,revenue,original_title,id
0,en,21.946943,81.0,30000000,"[{'id': 16, 'name': 'Animation'}, {'id': 35, '...",373554033.0,Toy Story,862
1,en,17.015539,104.0,65000000,"[{'id': 12, 'name': 'Adventure'}, {'id': 14, '...",262797249.0,Jumanji,8844
2,en,11.7129,101.0,0,"[{'id': 10749, 'name': 'Romance'}, {'id': 35, ...",0.0,Grumpier Old Men,15602
3,en,3.859495,127.0,16000000,"[{'id': 35, 'name': 'Comedy'}, {'id': 18, 'nam...",81452156.0,Waiting to Exhale,31357
4,en,8.387519,106.0,0,"[{'id': 35, 'name': 'Comedy'}]",76578911.0,Father of the Bride Part II,11862


In [13]:
#Tirando budgets diferente de 0 (budget ainda como string
df=df[df.budget != "0"]
df=df.drop([19730, 29503,35587])

In [14]:
#Transformando em float id e budget
df.id=df.id.astype('float64')
df.budget=df.budget.astype('float64')
df.dtypes

original_language     object
popularity            object
runtime              float64
budget               float64
genres                object
revenue              float64
original_title        object
id                   float64
dtype: object

In [15]:
#Ordenando em ordem crescente de acordo com o id, e filtrando valores 0 das variáveis float
df=df[(df.revenue >0) & (df.budget>0)  & (df.runtime>0)]
df=df.sort_values(['id'], ascending=[True])
df.head()

,original_language,popularity,runtime,budget,genres,revenue,original_title,id
17,en,9.026586,98.0,4000000.0,"[{'id': 80, 'name': 'Crime'}, {'id': 35, 'name...",4300000.0,Four Rooms,5.0
256,en,42.149697,121.0,11000000.0,"[{'id': 12, 'name': 'Adventure'}, {'id': 28, '...",775398007.0,Star Wars,11.0
6232,en,25.497794,100.0,94000000.0,"[{'id': 16, 'name': 'Animation'}, {'id': 10751...",940335536.0,Finding Nemo,12.0
351,en,48.307194,142.0,55000000.0,"[{'id': 35, 'name': 'Comedy'}, {'id': 18, 'nam...",677945399.0,Forrest Gump,13.0
2742,en,20.726578,122.0,15000000.0,"[{'id': 18, 'name': 'Drama'}]",356296601.0,American Beauty,14.0


In [16]:
#Tirando media dos ratings
imdb=imdb.sort_values(['movieId'], ascending=[True])
MediaRating=imdb.drop("userId",1,inplace=True)
MediaRating=imdb.drop("timestamp",1,inplace=True)
imdb.columns

Index(['movieId', 'rating'], dtype='object')

In [17]:
imdb.head()

,movieId,rating
9713,1,4.0
35933,1,1.5
52631,1,5.0
35983,1,2.5
12038,1,4.0


In [18]:
rating=pd.Series(imdb.groupby(['movieId'])['rating'].mean())

In [22]:
rating.reset_index().head()

,movieId,rating
0,1,3.872470
1,2,3.401869
2,3,3.161017
3,4,2.384615
4,5,3.267857
